<a href="https://colab.research.google.com/github/qrvdelossantos/CPE-311/blob/main/DELOSSANTOS_Hands_on_Activity_7_2_Webscraping_using_BeautifulSoup_and_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Rizaldy V. Delos Santos<br>
Date: 02/24/26<br>
Course & Section: CPE301-CPE22S3<br>
Instructor: Engr. Neal Matira

# **Data Gathering**

# **Example of gathering image data using webcam**

In [1]:
import cv2

# Start webcam
webcam = cv2.VideoCapture(0)

print("Press 's' to capture image")
print("Press 'q' to quit")

while True:
    try:
        # Read frame
        check, frame = webcam.read()
        print(check)  # True if webcam works

        if not check:
            print("Failed to access webcam")
            break

        # Show live camera
        cv2.imshow("Capturing", frame)

        key = cv2.waitKey(1)

        # Press 's' to save image
        if key == ord('s'):
            cv2.imwrite('saved_img.jpg', frame)
            print("Image saved!")

            # Convert to grayscale
            print("Converting RGB image to grayscale...")
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Resize to 28x28
            print("Resizing image to 28x28...")
            resized = cv2.resize(gray, (28, 28))

            # Save final processed image
            cv2.imwrite('saved_img-final.jpg', resized)
            print("Processed image saved!")

            # Show grayscale image
            cv2.imshow("Captured Image", resized)
            cv2.waitKey(1650)

            break

        # Press 'q' to quit
        elif key == ord('q'):
            print("Turning off camera.")
            break

    except KeyboardInterrupt:
        print("Interrupted by user.")
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()
print("Camera off. Program ended.")

Press 's' to capture image
Press 'q' to quit
False
Failed to access webcam
Camera off. Program ended.


# **Example of gathering voice data using microphone**

In [2]:
!pip3 install sounddevice

In [3]:
!pip3 install wavio

In [4]:
!pip3 install scipy

In [5]:
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (65.4 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.11) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur

In [6]:
# Import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
import wavio
import numpy as np

# Sampling frequency (Hz)
freq = 44100

# Recording duration (seconds)
duration = 5

print("Recording started...")

# Start recording (stereo: channels=2)
recording = sd.rec(int(duration * freq),
                   samplerate=freq,
                   channels=2)

# Wait until recording is finished
sd.wait()

print("Recording finished.")

# Convert float data to int16 (required for scipy write)
recording_int16 = np.int16(recording * 32767)

# Save using scipy
write("recording0.wav", freq, recording_int16)

# Save using wavio
wavio.write("recording1.wav", recording, freq, sampwidth=2)

print("Audio files saved successfully!")

Recording started...


PortAudioError: Error querying device -1

# **Web Scraping**

In [9]:
!pip install bs4

In [10]:
pip install requests

In [11]:
import requests
from bs4 import BeautifulSoup

def getdata(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    r = requests.get(url, headers=headers)
    return r.text

# Get HTML data
htmldata = getdata("https://www.google.com")

# Parse HTML
soup = BeautifulSoup(htmldata, "html.parser")

# Extract all image sources
for item in soup.find_all("img"):
    src = item.get("src")   # safer than item['src']
    if src:
        print(src)

/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png


In [7]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.3/510.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


# **Image Scraping using Selenium**

In [8]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import time
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Start driver
driver = webdriver.Chrome(options=chrome_options)

# Function to scroll page
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# Function to get image URLs
def get_image_urls(query, total_imgs):
    search_url = f"https://www.google.com/search?q={query}&tbm=isch"
    driver.get(search_url)

    img_urls = set()

    while len(img_urls) < total_imgs:
        scroll_to_end(driver)

        thumbnails = driver.find_elements(By.XPATH, "//img[contains(@class,'Q4LuWd')]")

        for img in thumbnails:
            try:
                img.click()
                time.sleep(1)

                images = driver.find_elements(By.CSS_SELECTOR, "img.n3VNCb")
                for image in images:
                    src = image.get_attribute("src")
                    if src and "http" in src:
                        img_urls.add(src)

                if len(img_urls) >= total_imgs:
                    break

            except:
                continue

        print(f"Collected: {len(img_urls)} images")

    return list(img_urls)[:total_imgs]

# Function to download images
def download_images(folder, urls):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i, url in enumerate(urls):
        try:
            image_content = requests.get(url).content
            with open(os.path.join(folder, f"image_{i}.jpg"), "wb") as f:
                f.write(image_content)
        except Exception as e:
            print(f"Could not download {url} - {e}")

# 🔹 Run scraper
urls = get_image_urls("Car", 10)
download_images("car_images", urls)

driver.quit()
print("Done!")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [39.2 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,913 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127

SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5b4b5769ceea <unknown>
#1 0x5b4b570abae5 <unknown>
#2 0x5b4b570e869b <unknown>
#3 0x5b4b570e3369 <unknown>
#4 0x5b4b5713357e <unknown>
#5 0x5b4b57132c6c <unknown>
#6 0x5b4b570f1cef <unknown>
#7 0x5b4b570f2ab1 <unknown>
#8 0x5b4b57661d64 <unknown>
#9 0x5b4b57664c51 <unknown>
#10 0x5b4b5764eb6a <unknown>
#11 0x5b4b5766581e <unknown>
#12 0x5b4b576356e0 <unknown>
#13 0x5b4b57689b18 <unknown>
#14 0x5b4b57689ce8 <unknown>
#15 0x5b4b5769b92e <unknown>
#16 0x782ae032dac3 <unknown>
